In [1]:
import os
import pandas as pd
import numpy as np
from datetime import date
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP")
from Defs import merge_SalesUnits
from Defs import merge_Mapping
from Defs import Period
from Defs import new_list
from Defs import export_from_RISKCUSTOM
from Defs import add_in_currency_column
from Defs import concat_columns
from Defs import export_from_WHWEEK
from Defs import CCY_tech_dict

In [2]:
print_to_excel = True
excel_tofolder_on_Z = True

In [3]:
excel_data = pd.read_excel('report_invest_31.07.2024.xlsx', sheet_name='snapshot', header=10)
today = '2024-07-31'

In [4]:
excel_data_work = excel_data[:].reset_index(drop=True)
excel_data_work = excel_data_work.rename(columns={'ПВК|Тип прогноз. данных':'currency', 'Unnamed: 10':'Cpty', 'Балансовая единица':'Comp_code'})
# merge
excel_data_work['Cpty_code'] = merge_Mapping(col='Cpty', df=excel_data_work)
excel_data_work['Cpty_segment'] = merge_SalesUnits(excel_data_work, col='Cpty_code', merge_col='ocpSegment')
excel_data_work['Comp_segment'] = merge_SalesUnits(excel_data_work, col='Comp_code', merge_col='ocpSegment')
excel_data_work['holding'] = merge_SalesUnits(excel_data_work,'Comp_code',merge_col='holding') 
# data
excel_data_work['КалендГод/Месяц'] = excel_data_work['КалендГод/Месяц'].astype(str)
excel_data_work['КалендГод/Месяц'] = pd.to_datetime(excel_data_work['КалендГод/Месяц'], format='%m.%Y')
# to usd
excel_data_work['Заказ на поставку'] = excel_data_work['Заказ на поставку'].astype('float64').fillna(0)
# excel_data_work = add_in_currency_column(excel_data_work,'USD','currency',False,'Заказ на поставку',today)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:99: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['SUEK RU' 'SUEK RU' 'SUEK RU' ... 'SGC' 'SGC' 'SGC']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  merge_data.loc[merge_data[f'{id_colmn}_merge'] != 'External', 'last_merge'] = merge_data.loc[merge_data[f'{id_colmn}_merge'] != 'External', f'{id_colmn}_merge']
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:99: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2

In [5]:
# excel_data_work[excel_data_work.Контрагент==1000079678]

In [6]:
# create version columns
pivot_index_list=['holding','Comp_segment','Comp_code','Завод',
       'ПЕ','СПП-элемент','Контрагент','Cpty_segment','Вид проекта','Профиль проекта',
       'Проект','Вид затрат','КалендГод/Месяц', 'Календарный год',
       'Календарный месяц','currency']
excel_data_work['Версия'] = excel_data_work['Версия'].apply(lambda x: 'Версия_0' + str(x))
excel_data_version_pivot = excel_data_work.pivot_table(index=pivot_index_list, columns='Версия', values='Заказ на поставку', aggfunc=sum).reset_index().sort_values(['Comp_code','Контрагент'])
new_cols = ['Версия_045', 'Версия_075', 'Версия_035', 'Версия_085']
# excel_data_version_pivot

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_20112\3410221329.py:7: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  excel_data_version_pivot = excel_data_work.pivot_table(index=pivot_index_list, columns='Версия', values='Заказ на поставку', aggfunc=sum).reset_index().sort_values(['Comp_code','Контрагент'])


In [7]:
excel_data_work['holding'].unique()

array(['EUROCHEM', 'SUEK'], dtype=object)

In [8]:
# compaire values
excel_data_version_pivot[new_cols] = excel_data_version_pivot[new_cols].fillna(0)
excel_data_version_pivot['Amount'] = np.nan
exit_data = pd.DataFrame()
for holding in ['EUROCHEM', 'SUEK']:
    holding_data = excel_data_version_pivot[excel_data_version_pivot.holding == holding]
    if holding == 'EUROCHEM':
    # Для БЕ из holding = Eurochem:
        # Если сумма платежей (значение поля " версия " = 075) меньше или равна сумме освоения (значение поля " версия " = 085), то берем платежи (значение поля " версия " = 075).
        holding_data.loc[holding_data['Версия_075']<=holding_data['Версия_085'], 'Amount'] = holding_data.loc[holding_data['Версия_075']<=holding_data['Версия_085'], 'Версия_075']
        # Если сумма платежей (значение поля " версия " = 075) больше суммы освоения (значение поля " версия " = 085), то берем сумму освоения (значение поля " версия " = 085).
        holding_data.loc[holding_data['Версия_075']>holding_data['Версия_085'], 'Amount'] = holding_data.loc[holding_data['Версия_075']>holding_data['Версия_085'], 'Версия_085']
    if holding == 'SUEK':
    # Для БЕ из holding = SUEK:
        # Если сумма платежей (значение поля " версия " = 045) меньше или равна сумме освоения (значение поля " версия " = 035), то берем платежи (значение поля " версия " = 045).
        holding_data.loc[holding_data['Версия_045']<=holding_data['Версия_035'], 'Amount'] = holding_data.loc[holding_data['Версия_045']<=holding_data['Версия_035'], 'Версия_045']
        # Если сумма платежей (значение поля " версия " = 045) больше суммы освоения (значение поля " версия " = 035), то берем сумму освоения (значение поля " версия " = 035).
        holding_data.loc[holding_data['Версия_045']>holding_data['Версия_035'], 'Amount'] = holding_data.loc[holding_data['Версия_045']>holding_data['Версия_035'], 'Версия_035']
    exit_data = pd.concat([exit_data, holding_data])

In [9]:
# exit_data[exit_data.Контрагент==1000079678]
# exit_data[exit_data.holding=='SUEK']

In [10]:
exit_data = add_in_currency_column(exit_data, CCY_to='USD', col_with_CCY='currency', col_with_VAL='Amount', DATE=today, date_is_column=False)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:99: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:168: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[f'Coef_to_{CCY_to}'] = df.date_CCY_from.replace(coef_dict).fillna(0)


In [11]:
exit_data.loc[exit_data.Coef_to_USD.isna(), 'currency'].drop_duplicates()
exit_data.loc[exit_data.Coef_to_USD.isna()]

Версия,holding,Comp_segment,Comp_code,Завод,ПЕ,СПП-элемент,Контрагент,Cpty_segment,Вид проекта,Профиль проекта,...,Календарный год,Календарный месяц,currency,Версия_035,Версия_045,Версия_075,Версия_085,Amount,Coef_to_USD,Amount_in_USD


In [12]:
### to excel
for holding in ['EUROCHEM', 'SUEK']:
    data = 0
    if print_to_excel == True:
        Output_file = f'{str(date.today())}_Invest_nv_{holding}.xlsx'
        if excel_tofolder_on_Z == True:
            Output_path = 'z:\\Anna_Klimova\\OCP\\Archive\\'
            Output_file = Output_path + Output_file
        data = exit_data[exit_data.holding==holding]
        data.to_excel(Output_file, sheet_name='Invest', index=False)

In [13]:
assert 2==1

AssertionError: 

In [ ]:
# compaire with old BD
old_BD_data_SUEK = pd.read_excel('2024-06-13_SUEK_Invest.xlsx',header=3).drop(0,axis=0) 
old_BD_data_SUEK_pivot = old_BD_data_SUEK.pivot_table(index='Unnamed: 5',values='Законтрактовано.2',aggfunc=sum).reset_index().rename(columns={'Unnamed: 5':'БЕ','Законтрактовано.2':'Сумма_итого_законтрактровнано'})
old_BD_data_EURO = pd.read_excel('2024-06-13_EUROCHEM_Invest.xlsx',header=3).drop(0,axis=0) 
old_BD_data_EURO_pivot = old_BD_data_EURO.pivot_table(index='Unnamed: 5',values='Законтрактовано.2',aggfunc=sum).reset_index().rename(columns={'Unnamed: 5':'БЕ','Законтрактовано.2':'Сумма_итого_законтрактровнано'})
old_BD_data_SUEK_pivot

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_22624\4027979718.py:3: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  old_BD_data_SUEK_pivot = old_BD_data_SUEK.pivot_table(index='Unnamed: 5',values='Законтрактовано.2',aggfunc=sum).reset_index().rename(columns={'Unnamed: 5':'БЕ','Законтрактовано.2':'Сумма_итого_законтрактровнано'})
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_22624\4027979718.py:5: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  old_BD_data_EURO_pivot = old_BD_data_EURO.pivot_table(index='Unnamed: 5',values='Законтрактовано.2',aggfunc=sum).reset_index().rename(columns={'Unnamed: 5':'БЕ','Зак

,БЕ,Сумма_итого_законтрактровнано
0,1100,156069493.09
1,1300,72101.36
2,1600,19216.02
3,1I00,104629.4
4,1M00,1584353.68
...,...,...
76,Y100,155526600.61
77,Y300,18343444.65
78,Y500,26009.73
79,Y600,0


In [ ]:
compaire_to_print = df_to_print.pivot_table(index='Балансовая единица',aggfunc=sum,values='amount').reset_index().merge(old_BD_data_SUEK_pivot,how='outer',left_on='Балансовая единица',right_on='БЕ').sort_values('amount',ascending=False)
compaire_to_print = compaire_to_print.fillna(0)
compaire_to_print['is_equal'] = compaire_to_print.amount.astype(int)==compaire_to_print['Сумма_итого_законтрактровнано'].astype(int)
compaire_to_print = compaire_to_print[compaire_to_print.Сумма_итого_законтрактровнано!=0]
if print_to_excel==True:
    Output_file = f'{str(date.today())}_Invest_from_report_invest_30.05.2024.xlsx'
    compaire_to_print.to_excel(Output_file,sheet_name='compaire',index=False)
    new_list(df_to_print,Output_file,'data')

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_22624\307249084.py:1: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  compaire_to_print = df_to_print.pivot_table(index='Балансовая единица',aggfunc=sum,values='amount').reset_index().merge(old_BD_data_SUEK_pivot,how='outer',left_on='Балансовая единица',right_on='БЕ').sort_values('amount',ascending=False)
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_22624\307249084.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  compaire_to_print = compaire_to_print.fillna(0)


In [ ]:
new_list(excel_data_version_pivot[(excel_data_version_pivot.Версия_045==0)&(excel_data_version_pivot.Версия_075!=0)],Output_file,'errors')